# Mr Haulage - Exploratory Data Analysis
## Author: Lottie Jane Pollard

*"Data matters. Data is the way we measure progress." - Hilary Mason*
*"Without data, you're just another person with an opinion." - W. Edwards Deming*

In [1]:
# listing libraries to import

import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import CLEAN dataset

mr_haulage_df = pd.read_csv(
    '/Users/lottiejanepollare/Library/Mobile Documents/com~apple~CloudDocs/CV, Profiles, Interviews & Job Applications/applications/techmodal_analyst_data_engineer/20230825_Analyst_case_study_submission_Lottie_Jane_Pollard/datasets/cleansed_mr_haulage_order_details.csv')

# add data types in from previous metadata

metadata_df = pd.read_excel('/Users/lottiejanepollare/Library/Mobile Documents/com~apple~CloudDocs/CV, Profiles, Interviews & Job Applications/applications/techmodal_analyst_data_engineer/20230825_Analyst_case_study_submission_Lottie_Jane_Pollard/datasets/metadata.xlsx')
metadata_dict = metadata_df.set_index('column_name')['datatype'].to_dict()
mr_haulage_df = mr_haulage_df.astype(metadata_dict)

# configure display settings: display all columns regardless of df width, disable wrapping columns to display entire field, no truncating columns, display an English date format
pd.set_option('display.max.columns', None, 'display.width', None, 'display.max.colwidth', None, 'display.date_dayfirst', True)

# show the head of df to see what I'm working with
mr_haulage_df.head(50)

,order_id,customer_id,order_date,order_time,item_serial,box_type,delivery_region,distance_(miles),order_week,order_month,order_year,financial_quarter
0,1097342,733603,2021-08-22,00:14,30351,small,south_east,70,33,August,2021,Q3
1,1097343,405061,2021-08-22,07:08,17634,small,greater_london,32,33,August,2021,Q3
2,1097344,842139,2021-08-22,10:15,25598,small,south_west,190,33,August,2021,Q3
3,1097345,211806,2021-08-22,17:05,10104,small,south_west,85,33,August,2021,Q3
4,1097346,103222,2021-08-22,23:48,3252,small,greater_london,43,33,August,2021,Q3
5,1097347,603400,2021-08-22,23:57,62831,small,greater_london,33,33,August,2021,Q3
6,1097354,862722,2021-08-23,21:56,32892,small,south_wales,171,34,August,2021,Q3
7,1097353,697945,2021-08-23,20:34,23747,small,south_east,67,34,August,2021,Q3
8,1097352,870782,2021-08-23,19:03,20624,small,south_east,72,34,August,2021,Q3
9,1097350,239710,2021-08-23,11:49,99590,large,north_east,210,34,August,2021,Q3


In [3]:
# check metadata & datatypes have loaded correctly
mr_haulage_df.dtypes

order_id                      int64
customer_id                   int64
order_date           datetime64[ns]
order_time                   object
item_serial                   int64
box_type                   category
delivery_region            category
distance_(miles)              int64
order_week                   UInt32
order_month                  object
order_year                    int32
financial_quarter            object
dtype: object

## Let's start our cost analysis by adding a column for revenue per box to allow for some Pandas aggregation & visualisations

In [4]:
# first, we will create the values based on if/else criteria (small box = £20 & large box = £100)
initial_values = [20.00 if box_type == 'small' else 100.00 if box_type == 'large' else 0 for box_type in mr_haulage_df['box_type']]

# then, insert a new column with the above initial values, I'll insert the new column, 'order_revenue' after 'item_serial' at index 3
mr_haulage_df.insert(loc=4, column='order_revenue', value=initial_values)

# set the data type for 'order_revenue' to float64 for monetary value
mr_haulage_df['order_revenue'] = mr_haulage_df['order_revenue'].astype('float64')
pd.set_option('display.float_format', '{:.2f}'.format)

mr_haulage_df

,order_id,customer_id,order_date,order_time,order_revenue,item_serial,box_type,delivery_region,distance_(miles),order_week,order_month,order_year,financial_quarter
0,1097342,733603,2021-08-22,00:14,20.00,30351,small,south_east,70,33,August,2021,Q3
1,1097343,405061,2021-08-22,07:08,20.00,17634,small,greater_london,32,33,August,2021,Q3
2,1097344,842139,2021-08-22,10:15,20.00,25598,small,south_west,190,33,August,2021,Q3
3,1097345,211806,2021-08-22,17:05,20.00,10104,small,south_west,85,33,August,2021,Q3
4,1097346,103222,2021-08-22,23:48,20.00,3252,small,greater_london,43,33,August,2021,Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1099335,216509,2023-04-09,06:40,20.00,4716,small,greater_london,8,14,April,2023,Q2
1996,1099339,710623,2023-04-10,11:32,20.00,387,small,south_west,300,15,April,2023,Q2
1997,1099340,932977,2023-04-10,17:54,100.00,80608,large,south_east,48,15,April,2023,Q2
1998,1099338,103222,2023-04-10,01:02,20.00,29091,small,south_west,233,15,April,2023,Q2


In [42]:
# let's look at the 'distance_(miles)' in relation to the region. This will give me an idea of where the main distribution depot is
avg_distance_per_region = mr_haulage_df.groupby('delivery_region')['distance_(miles)'].mean().reset_index().sort_values(by='distance_(miles)')

print(f"The main distribution depot is most likely to be in {avg_distance_per_region['delivery_region'].iloc[0]}")
print(f"--------------------------------------------------------")
avg_distance_per_region

The main distribution depot is most likely to be in greater_london
--------------------------------------------------------


,delivery_region,distance_(miles)
1,greater_london,26.91
4,south_east,60.23
0,east_midlands,82.70
7,west_midlands,119.23
6,south_west,179.88
5,south_wales,195.53
3,north_wales,213.87
2,north_east,233.84


In [43]:
# let's look at the most popular customer_id's

# Number of unique customers
number_of_unique_customers = mr_haulage_df['customer_id'].nunique()

# Orders per customer
orders_per_customer = mr_haulage_df.groupby('customer_id').size().reset_index(name='number_of_orders').sort_values(by='number_of_orders', ascending=False)

# Repeat customers
repeat_customers = orders_per_customer[orders_per_customer['number_of_orders'] > 1]

print(f"We have {number_of_unique_customers} unique customers.")
print(f"--------------------------------------------------------")
print(f"Of our {number_of_unique_customers} customers, only {len(repeat_customers)} of them are repeat customers.")
print(f"--------------------------------------------------------")

orders_per_customer.head(len(repeat_customers))

We have 1792 unique customers.
--------------------------------------------------------
Of our 1792 customers, only 4 of them are repeat customers.
--------------------------------------------------------


,customer_id,number_of_orders
8,103222,106
12,104011,77
1,100131,27
299,246796,2


In [21]:
# Get the customer IDs of the first 4 repeat customers
top_4_repeat_customer_ids = repeat_customers['customer_id'].head(4).values

# Filter the DataFrame for those top 4 repeat customer IDs
filtered_df = mr_haulage_df[mr_haulage_df['customer_id'].isin(top_4_repeat_customer_ids)]

# Find the most common delivery region for each of those top 4 customer IDs
for customer_id in top_4_repeat_customer_ids:
    most_common_region = filtered_df[filtered_df['customer_id'] == customer_id]['delivery_region'].value_counts().idxmax()
    print(f"The most common delivery region for customer ID {customer_id} is {most_common_region}.")
    print(f"--------------------------------------------------------")


The most common delivery region for customer ID 103222 is greater_london.
--------------------------------------------------------
The most common delivery region for customer ID 104011 is greater_london.
--------------------------------------------------------
The most common delivery region for customer ID 100131 is greater_london.
--------------------------------------------------------
The most common delivery region for customer ID 246796 is south_east.
--------------------------------------------------------


In [16]:
# let's look at the most popular delivery regions
most_popular_delivery_regions_by_orders = mr_haulage_df.groupby('delivery_region').size().reset_index(name='order_count').sort_values('order_count', ascending=False)

# Calculate the total number of orders
total_orders = most_popular_delivery_regions_by_orders['order_count'].sum()

# Add a new column for the percentage of total orders
most_popular_delivery_regions_by_orders['percentage_of_total_orders'] = (most_popular_delivery_regions_by_orders['order_count'] / total_orders) * 100

most_popular_delivery_regions_by_orders

,delivery_region,order_count,percentage_of_total_orders
1,greater_london,629,31.45
4,south_east,475,23.75
6,south_west,309,15.45
0,east_midlands,199,9.95
5,south_wales,132,6.60
2,north_east,108,5.40
7,west_midlands,96,4.80
3,north_wales,52,2.60


In [17]:
# let's look at the most popular delivery regions by total customers
most_popular_delivery_regions_by_customers = mr_haulage_df.groupby('delivery_region')['customer_id'].nunique().reset_index(name='unique_customer_count').sort_values('unique_customer_count', ascending=False)

# Calculate the total number of customers
total_customers = most_popular_delivery_regions_by_customers['unique_customer_count'].sum()

# Add a new column for the percentage of total customers
most_popular_delivery_regions_by_customers['percentage_of_total_customers'] = (most_popular_delivery_regions_by_customers['unique_customer_count'] / total_customers) * 100

most_popular_delivery_regions_by_customers

,delivery_region,unique_customer_count,percentage_of_total_customers
1,greater_london,563,31.05
4,south_east,436,24.05
6,south_west,276,15.22
0,east_midlands,171,9.43
5,south_wales,124,6.84
2,north_east,102,5.63
7,west_midlands,91,5.02
3,north_wales,50,2.76


In [44]:
# first, let's look at the historical daily demand for each box size per region
estimated_daily_demand = mr_haulage_df.groupby(['order_date', 'delivery_region', 'box_type']).size().reset_index(name='box_count')

# Filter out rows where count is 0
estimated_daily_demand = estimated_daily_demand[estimated_daily_demand['box_count'] > 0]

estimated_daily_demand

,order_date,delivery_region,box_type,box_count
3,2021-08-22,greater_london,small,3
9,2021-08-22,south_east,small,1
13,2021-08-22,south_west,small,2
18,2021-08-23,greater_london,large,1
20,2021-08-23,north_east,large,1
...,...,...,...,...
9523,2023-04-09,greater_london,small,2
9533,2023-04-09,south_west,small,1
9539,2023-04-10,greater_london,small,1
9544,2023-04-10,south_east,large,1


# OPTION 1 - PRIORITISING LARGE TRUCKS TO ALLOW FOR GROWTH
## by opting to purchase large trucks instead of small (in cases where you have surplus 1,2,3 small boxes - not enough to fill a large truck) you are allowing for growth of an extra small box & giving yourself the flexibility to utilise the same vehicle for large or small boxes)

In [46]:
# let's copy the estimated daily demand to calculate for OPTION 1
daily_demand_op_1 = estimated_daily_demand.copy()

In [47]:
# Initialize an empty list to store the aggregated records
aggregated_data_daily = []

# Loop through each unique combination of delivery_region and order_date
for (region, date) in daily_demand_op_1.groupby(['delivery_region', 'order_date']).groups.keys():
    region_date_data = daily_demand_op_1[(daily_demand_op_1['delivery_region'] == region) & (daily_demand_op_1['order_date'] == date)]

    # Initialize counters for the number of trucks
    daily_small_trucks = 0
    daily_large_trucks = 0

    # Calculate the number of small trucks needed
    if 'small' in region_date_data['box_type'].values:
        daily_small_box_count = region_date_data.loc[region_date_data['box_type'] == 'small', 'box_count'].values[0]

        # Convert sets of 4 small boxes to a large truck
        daily_large_trucks += daily_small_box_count // 4
        remaining_small_boxes = daily_small_box_count % 4

        # If there are 3 remaining small boxes, use a large truck
        if remaining_small_boxes == 3:
            daily_large_trucks += 1
        # If there are 2 remaining small boxes, use two small trucks
        if remaining_small_boxes == 2:
            daily_small_trucks += 2
        # If there is 1 remaining small box, use a small truck
        elif remaining_small_boxes == 1:
            daily_small_trucks += 1

    # Calculate the number of large trucks needed
    if 'large' in region_date_data['box_type'].values:
        daily_large_box_count = region_date_data.loc[region_date_data['box_type'] == 'large', 'box_count'].values[0]
        daily_large_trucks += daily_large_box_count

    # Append the aggregated information to the list
    aggregated_data_daily.append([date, region, daily_small_trucks, daily_large_trucks])

# Convert the list to a DataFrame
aggregated_df_daily = pd.DataFrame(aggregated_data_daily, columns=['order_date', 'delivery_region', 'daily_small_trucks_needed', 'daily_large_trucks_needed'])
aggregated_df_daily = aggregated_df_daily.sort_values(by='order_date', ascending=True)

aggregated_df_daily

,order_date,delivery_region,daily_small_trucks_needed,daily_large_trucks_needed
706,2021-08-22,south_east,1,0
1169,2021-08-22,south_west,2,0
171,2021-08-22,greater_london,0,1
1170,2021-08-23,south_west,1,0
1403,2021-08-23,west_midlands,0,1
...,...,...,...,...
554,2023-04-09,greater_london,2,0
1401,2023-04-09,south_west,1,0
1402,2023-04-10,south_west,2,0
555,2023-04-10,greater_london,1,0


In [48]:
# let's aggregate the count by date
aggregated_df_grouped = aggregated_df_daily.groupby('order_date').agg({'daily_small_trucks_needed': 'sum', 'daily_large_trucks_needed': 'sum'}).reset_index()
aggregated_df_grouped

,order_date,daily_small_trucks_needed,daily_large_trucks_needed
0,2021-08-22,3,1
1,2021-08-23,4,3
2,2021-08-24,7,3
3,2021-08-25,6,1
4,2021-08-26,3,3
...,...,...,...
592,2023-04-06,3,1
593,2023-04-07,4,1
594,2023-04-08,2,1
595,2023-04-09,3,0


In [49]:
# now, to see the maximum, we'd need to service the contract efficiently

max_small_trucks = aggregated_df_grouped['daily_small_trucks_needed'].max()
max_large_trucks = aggregated_df_grouped['daily_large_trucks_needed'].max()

print(f"The total number of Small Trucks to purchase based on the historical dataset provided is: {max_small_trucks}.")
print(f"--------------------------------------------------------")
print(f"The total number of Large Trucks to purchase based on the historical dataset provided is: {max_large_trucks}.")
print(f"--------------------------------------------------------")

The total number of Small Trucks to purchase based on the historical dataset provided is: 8.
--------------------------------------------------------
The total number of Large Trucks to purchase based on the historical dataset provided is: 5.
--------------------------------------------------------


# OPTION 2 - PRIORITISING SMALL TRUCKS TO MINIMISE EXPENDITURE
## changing the function to increment small trucks more over large trucks (3 small boxes = 3 small trucks) only opting to increment by large truck when capacity (4 small boxes) is reached

In [50]:
# let's copy the estimated daily demand to calculate for OPTION 1
daily_demand_op_2 = estimated_daily_demand.copy()

In [52]:
# Initialize an empty list to store the aggregated records
aggregated_data_daily_op_2 = []

# Loop through each unique combination of delivery_region and order_date
for (region, date) in daily_demand_op_2.groupby(['delivery_region', 'order_date']).groups.keys():
    region_date_data_op_2 = daily_demand_op_2[(daily_demand_op_2['delivery_region'] == region) & (daily_demand_op_2['order_date'] == date)]

    # Initialize counters for the number of trucks
    daily_small_trucks_op_2 = 0
    daily_large_trucks_op_2 = 0

    # Calculate the number of small trucks needed
    if 'small' in region_date_data_op_2['box_type'].values:
        daily_small_box_count_op_2 = region_date_data_op_2.loc[region_date_data_op_2['box_type'] == 'small', 'box_count'].values[0]

        # Convert sets of 4 small boxes to a large truck
        daily_large_trucks_op_2 += daily_small_box_count_op_2 // 4
        remaining_small_boxes_op_2 = daily_small_box_count_op_2 % 4

        # If there are 1, 2, or 3 remaining small boxes, increment small trucks by that number
        if remaining_small_boxes_op_2 in [1, 2, 3]:
            daily_small_trucks_op_2 += remaining_small_boxes_op_2

    # Calculate the number of large trucks needed
    if 'large' in region_date_data_op_2['box_type'].values:
        daily_large_box_count_op_2 = region_date_data_op_2.loc[region_date_data_op_2['box_type'] == 'large', 'box_count'].values[0]
        daily_large_trucks_op_2 += daily_large_box_count_op_2

    # Append the aggregated information to the list
    aggregated_data_daily_op_2.append([date, region, daily_small_trucks_op_2, daily_large_trucks_op_2])

# Convert the list to a DataFrame
aggregated_df_daily_op_2 = pd.DataFrame(aggregated_data_daily_op_2, columns=['order_date', 'delivery_region', 'daily_small_trucks_needed', 'daily_large_trucks_needed'])
aggregated_df_daily_op_2 = aggregated_df_daily_op_2.sort_values(by='order_date', ascending=True)

aggregated_df_daily_op_2

,order_date,delivery_region,daily_small_trucks_needed,daily_large_trucks_needed
706,2021-08-22,south_east,1,0
1169,2021-08-22,south_west,2,0
171,2021-08-22,greater_london,3,0
1170,2021-08-23,south_west,1,0
1403,2021-08-23,west_midlands,0,1
...,...,...,...,...
554,2023-04-09,greater_london,2,0
1401,2023-04-09,south_west,1,0
1402,2023-04-10,south_west,2,0
555,2023-04-10,greater_london,1,0


In [53]:
# let's aggregate the count by date
aggregated_df_grouped_op_2 = aggregated_df_daily_op_2.groupby('order_date').agg({'daily_small_trucks_needed': 'sum', 'daily_large_trucks_needed': 'sum'}).reset_index()
aggregated_df_grouped_op_2

,order_date,daily_small_trucks_needed,daily_large_trucks_needed
0,2021-08-22,6,0
1,2021-08-23,4,3
2,2021-08-24,7,3
3,2021-08-25,6,1
4,2021-08-26,3,3
...,...,...,...
592,2023-04-06,3,1
593,2023-04-07,7,0
594,2023-04-08,2,1
595,2023-04-09,3,0


In [63]:
# CHECKING AVERAGES...... WOULDN'T WORK AS YOU'D HAVE A HUGE DEFICIT

avg_small_trucks_needed_op_2 = math.ceil(aggregated_df_grouped_op_2['daily_small_trucks_needed'].mean())
avg_large_trucks_needed_op_2 = math.ceil(aggregated_df_grouped_op_2['daily_large_trucks_needed'].mean())

avg_small_trucks_needed_op_2, avg_large_trucks_needed_op_2

(3, 1)

In [54]:
# now, to see the maximum, we'd need to service the contract efficiently

max_small_trucks_op_2 = aggregated_df_grouped_op_2['daily_small_trucks_needed'].max()
max_large_trucks_op_2 = aggregated_df_grouped_op_2['daily_large_trucks_needed'].max()

print(f"The total number of Small Trucks to purchase based on the historical dataset provided is: {max_small_trucks_op_2}.")
print(f"--------------------------------------------------------")
print(f"The total number of Large Trucks to purchase based on the historical dataset provided is: {max_large_trucks_op_2}.")
print(f"--------------------------------------------------------")

The total number of Small Trucks to purchase based on the historical dataset provided is: 8.
--------------------------------------------------------
The total number of Large Trucks to purchase based on the historical dataset provided is: 5.
--------------------------------------------------------


# CONCLUSION
### Both options proved the same

In [ ]:
# orders per year & y/y
# orders per month & m/m

In [ ]:
# NOT MY CODE - working out averages (compare with my code)

import pandas as pd

# Load the CSV file
df = pd.read_csv('path_to_your_file.csv')

# Calculate the daily demand for small and large boxes in each region
daily_demand = df.groupby(['order_date', 'delivery_region', 'box_type']).size().reset_index(name='count')

# Calculate the average daily demand per region and per box type
avg_daily_demand = daily_demand.groupby(['delivery_region', 'box_type']).agg({'count': 'mean'}).reset_index()

# Rename the column to be more descriptive
avg_daily_demand.rename(columns={'count': 'avg_daily_demand'}, inplace=True)

print(avg_daily_demand)

In [ ]:
import math

def calculate_trucks_needed(avg_daily_demand):
    # Initialize variables to keep track of total trucks needed
    total_small_trucks = 0
    total_large_trucks = 0

    # Loop through each region
    for region in avg_daily_demand['delivery_region'].unique():
        # Filter average daily demand for the current region
        region_data = avg_daily_demand[avg_daily_demand['delivery_region'] == region]

        # Initialize variables to keep track of trucks needed for the current region
        region_small_trucks = 0
        region_large_trucks = 0

        # Calculate trucks needed for small boxes
        small_boxes_demand = region_data.loc[region_data['box_type'] == 'small', 'avg_daily_demand'].values[0]
        region_large_trucks += math.ceil(small_boxes_demand / 4)

        # Check if an additional small truck is needed for remaining small boxes
        if small_boxes_demand % 4 != 0:
            region_small_trucks += 1

        # Calculate trucks needed for large boxes
        large_boxes_demand = region_data.loc[region_data['box_type'] == 'large', 'avg_daily_demand'].values[0]
        region_large_trucks += large_boxes_demand

        # Update total trucks needed
        total_small_trucks += region_small_trucks
        total_large_trucks += region_large_trucks

        print(f"For region {region}: {region_small_trucks} small trucks and {region_large_trucks} large trucks are needed.")

    print(f"Total small trucks needed: {total_small_trucks}")
    print(f"Total large trucks needed: {total_large_trucks}")

calculate_trucks_needed(avg_daily_demand)